In [2]:
from google.colab import files
uploaded = files.upload()

Saving adult.csv to adult.csv


In [3]:
import pandas as pd

df = pd.read_csv("adult.csv")
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [4]:
df.shape

(32561, 15)

In [5]:
# Handling missing values:
df = df.replace("?", pd.NA)
df.isnull().sum()

,0
age,0
workclass,1836
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,1843
relationship,0
race,0
sex,0


In [6]:
df = df.dropna()
df.shape

(30162, 15)

In [7]:
#x and y split

X = df.drop("income", axis=1)
y = df["income"]

X.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States
5,34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States
6,38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States


In [8]:
y.head()

,income
1,<=50K
3,<=50K
4,<=50K
5,<=50K
6,<=50K


In [9]:
# encoding target variable
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

y[:5]

array([0, 0, 0, 0, 0])

In [10]:
# Checking class balance
import numpy as np

unique, counts = np.unique(y, return_counts=True)
dict(zip(unique, counts))

{np.int64(0): np.int64(22654), np.int64(1): np.int64(7508)}

In [11]:
X = pd.get_dummies(X, drop_first=True)
X.head()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native.country_Portugal,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia
1,82,132870,9,0,4356,18,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
3,54,140359,4,0,3900,40,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
4,41,264663,10,0,3900,40,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
5,34,216864,9,0,3770,45,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
6,38,150601,6,0,3770,40,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False


In [12]:
X.shape

(30162, 96)

In [13]:
#train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [14]:
print(X_train.shape)
print(X_test.shape)

(24129, 96)
(6033, 96)


In [15]:
# Scaling data
# Require for Logistic regression and KNN
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
#1. Logistic regression
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)

lr.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [17]:
#1. Prediction by logistic regression
y_pred_lr = lr.predict(X_test_scaled)
y_prob_lr = lr.predict_proba(X_test_scaled)[:, 1]


In [18]:
#1. Metrics for logistic regression

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef

print("Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Precision:", precision_score(y_test, y_pred_lr))
print("Recall:", recall_score(y_test, y_pred_lr))
print("F1 Score:", f1_score(y_test, y_pred_lr))
print("AUC:", roc_auc_score(y_test, y_prob_lr))
print("MCC:", matthews_corrcoef(y_test, y_pred_lr))

Accuracy: 0.8547986076578816
Precision: 0.7504
Recall: 0.6245006657789614
F1 Score: 0.6816860465116279
AUC: 0.9132284740040573
MCC: 0.59281833513439


In [19]:
#2. Decision tree model- with unscaled data

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)

dt.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [20]:
#2. Predictions woth Decision tree model

y_pred_dt = dt.predict(X_test)
y_prob_dt = dt.predict_proba(X_test)[:, 1]

In [21]:
#2. Metrics for Decision tree model

print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Precision:", precision_score(y_test, y_pred_dt))
print("Recall:", recall_score(y_test, y_pred_dt))
print("F1 Score:", f1_score(y_test, y_pred_dt))
print("AUC:", roc_auc_score(y_test, y_prob_dt))
print("MCC:", matthews_corrcoef(y_test, y_pred_dt))

Accuracy: 0.8155146693187469
Precision: 0.6299265197060788
Recall: 0.6278295605858855
F1 Score: 0.6288762920973658
AUC: 0.7527803728773612
MCC: 0.5061251518951079


In [22]:
#3. KNN model with unscaled data


from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train_scaled, y_train)


KNeighborsClassifier()

In [23]:
#3. Predicting with KNN model
y_pred_knn = knn.predict(X_test_scaled)
y_prob_knn = knn.predict_proba(X_test_scaled)[:, 1]

In [24]:
#3. Metrics for KNN model:
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("Precision:", precision_score(y_test, y_pred_knn))
print("Recall:", recall_score(y_test, y_pred_knn))
print("F1 Score:", f1_score(y_test, y_pred_knn))
print("AUC:", roc_auc_score(y_test, y_prob_knn))
print("MCC:", matthews_corrcoef(y_test, y_pred_knn))

Accuracy: 0.8257914801922758
Precision: 0.6666666666666666
Recall: 0.6005326231691078
F1 Score: 0.6318739054290718
AUC: 0.8497421667747646
MCC: 0.5193911138654854


In [25]:
#4. Naive Bayes model on unscaled data

from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_train, y_train)

GaussianNB()

In [26]:
#4. Prediction using Naive Bayes model
y_pred_nb = nb.predict(X_test)
y_prob_nb = nb.predict_proba(X_test)[:, 1]

In [27]:
#4. Metrics for Naive Bayes model

print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print("Precision:", precision_score(y_test, y_pred_nb))
print("Recall:", recall_score(y_test, y_pred_nb))
print("F1 Score:", f1_score(y_test, y_pred_nb))
print("AUC:", roc_auc_score(y_test, y_prob_nb))
print("MCC:", matthews_corrcoef(y_test, y_pred_nb))

Accuracy: 0.7909829272335488
Precision: 0.6680613668061367
Recall: 0.3189081225033289
F1 Score: 0.4317260027039207
AUC: 0.8368563830584455
MCC: 0.3559453464690397


In [28]:
#5. Random Fores model on unscaled data

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=20,
    max_depth=10,
    random_state=42
)

rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=20, random_state=42)

In [29]:
#5. Predictions using Random Forest Model

y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]

In [30]:
#5. Metrics of Random Forest model
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf))
print("Recall:", recall_score(y_test, y_pred_rf))
print("F1 Score:", f1_score(y_test, y_pred_rf))
print("AUC:", roc_auc_score(y_test, y_prob_rf))
print("MCC:", matthews_corrcoef(y_test, y_pred_rf))

Accuracy: 0.8571191778551301
Precision: 0.8180914512922465
Recall: 0.547936085219707
F1 Score: 0.6562998405103668
AUC: 0.9160739406973295
MCC: 0.58878316764407


In [31]:
#6. XGBoost model

!pip install xgboost
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=100,
    random_state=42,
    eval_metric='logloss'
)

xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [32]:
#6. Prediction using XGBoost

y_pred_xgb = xgb.predict(X_test)
y_prob_xgb = xgb.predict_proba(X_test)[:, 1]

In [33]:
#6. Metrics using XGBoost

print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("Precision:", precision_score(y_test, y_pred_xgb))
print("Recall:", recall_score(y_test, y_pred_xgb))
print("F1 Score:", f1_score(y_test, y_pred_xgb))
print("AUC:", roc_auc_score(y_test, y_prob_xgb))
print("MCC:", matthews_corrcoef(y_test, y_pred_xgb))

Accuracy: 0.8722028841372451
Precision: 0.7903097696584591
Recall: 0.6624500665778962
F1 Score: 0.720753350235422
AUC: 0.9338400708126677
MCC: 0.6429063390915257


In [34]:
# Comparison Table for README

results = {
    "Logistic Regression": {
        "Accuracy": accuracy_score(y_test, y_pred_lr),
        "AUC": roc_auc_score(y_test, y_prob_lr),
        "Precision": precision_score(y_test, y_pred_lr),
        "Recall": recall_score(y_test, y_pred_lr),
        "F1": f1_score(y_test, y_pred_lr),
        "MCC": matthews_corrcoef(y_test, y_pred_lr)
    },
    "Decision Tree": {
        "Accuracy": accuracy_score(y_test, y_pred_dt),
        "AUC": roc_auc_score(y_test, y_prob_dt),
        "Precision": precision_score(y_test, y_pred_dt),
        "Recall": recall_score(y_test, y_pred_dt),
        "F1": f1_score(y_test, y_pred_dt),
        "MCC": matthews_corrcoef(y_test, y_pred_dt)
    },
    "KNN": {
        "Accuracy": accuracy_score(y_test, y_pred_knn),
        "AUC": roc_auc_score(y_test, y_prob_knn),
        "Precision": precision_score(y_test, y_pred_knn),
        "Recall": recall_score(y_test, y_pred_knn),
        "F1": f1_score(y_test, y_pred_knn),
        "MCC": matthews_corrcoef(y_test, y_pred_knn)
    },
    "Naive Bayes": {
        "Accuracy": accuracy_score(y_test, y_pred_nb),
        "AUC": roc_auc_score(y_test, y_prob_nb),
        "Precision": precision_score(y_test, y_pred_nb),
        "Recall": recall_score(y_test, y_pred_nb),
        "F1": f1_score(y_test, y_pred_nb),
        "MCC": matthews_corrcoef(y_test, y_pred_nb)
    },
    "Random Forest": {
        "Accuracy": accuracy_score(y_test, y_pred_rf),
        "AUC": roc_auc_score(y_test, y_prob_rf),
        "Precision": precision_score(y_test, y_pred_rf),
        "Recall": recall_score(y_test, y_pred_rf),
        "F1": f1_score(y_test, y_pred_rf),
        "MCC": matthews_corrcoef(y_test, y_pred_rf)
    },
    "XGBoost": {
        "Accuracy": accuracy_score(y_test, y_pred_xgb),
        "AUC": roc_auc_score(y_test, y_prob_xgb),
        "Precision": precision_score(y_test, y_pred_xgb),
        "Recall": recall_score(y_test, y_pred_xgb),
        "F1": f1_score(y_test, y_pred_xgb),
        "MCC": matthews_corrcoef(y_test, y_pred_xgb)
    }
}

results_df = pd.DataFrame(results).T
results_df

,Accuracy,AUC,Precision,Recall,F1,MCC
Logistic Regression,0.854799,0.913228,0.750400,0.624501,0.681686,0.592818
Decision Tree,0.815515,0.752780,0.629927,0.627830,0.628876,0.506125
KNN,0.825791,0.849742,0.666667,0.600533,0.631874,0.519391
Naive Bayes,0.790983,0.836856,0.668061,0.318908,0.431726,0.355945
Random Forest,0.857119,0.916074,0.818091,0.547936,0.656300,0.588783
XGBoost,0.872203,0.933840,0.790310,0.662450,0.720753,0.642906


In [35]:
# Saving the project for Github

import joblib
import os

os.makedirs("models", exist_ok=True)

joblib.dump(lr, "models/logistic.pkl")
joblib.dump(dt, "models/decision_tree.pkl")
joblib.dump(knn, "models/knn.pkl")
joblib.dump(nb, "models/naive_bayes.pkl")
joblib.dump(rf, "models/random_forest.pkl")
joblib.dump(xgb, "models/xgboost.pkl")
joblib.dump(scaler, "models/scaler.pkl")

['models/scaler.pkl']

In [36]:
# downloading file for github

!zip -r project_files.zip models

  adding: models/ (stored 0%)
  adding: models/decision_tree.pkl (deflated 85%)
  adding: models/scaler.pkl (deflated 34%)
  adding: models/xgboost.pkl (deflated 68%)
  adding: models/random_forest.pkl (deflated 74%)
  adding: models/knn.pkl (deflated 96%)
  adding: models/naive_bayes.pkl (deflated 40%)
  adding: models/logistic.pkl (deflated 16%)


In [37]:
from google.colab import files
files.download("project_files.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
joblib.dump(X_train.columns.tolist(), "models/training_columns.pkl")

['models/training_columns.pkl']

In [39]:
from google.colab import files
files.download("models/training_columns.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
# Create test dataset dataframe
test_df = X_test.copy()
test_df["income"] = y_test

test_df.to_csv("adult_test_data.csv", index=False)

In [42]:
from google.colab import files
files.download("adult_test_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>